# Install Dkube Pipelines SDK

In [ ]:
!pip install git+https://github.com/oneconvergence/dkube.git@pipelines-support --upgrade

# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [ ]:
%env RELEASE_VERSION=0.1.18
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

# Import kfp pkgs

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [ ]:
client = kfp.Client()
client.list_experiments()

# Create a Dkube MNIST experiment

In [ ]:
mnist_experiment = client.create_experiment(name='Dkube - Mnist pl')

# Define e2e MNIST Pipeline with Dkube components

In [ ]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

import sys
sys.path.insert(0,'/opt/conda/lib/python3.7/site-packages')
from dkube.pipelines import *

dkube_training_op            = DkubeTrainOp(name='mnist-training')
dkube_serving_op             = DkubeServeOp(name='mnist-serving')
dkube_viewer_op              = DkubeViewerOp(name='mnist-inference')

@dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist digits pipeline with dkube components'
)
def d3pipeline(
    #In notebook ACCESS_TOKEN is automatically picked up from env variable
    auth_token  = os.getenv("ACCESS_TOKEN"),
    #By default tf v1.12 image is used here, v1.10, v1.11 or v1.13 can be used. 
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'docker.io/ocdr/dkube-datascience-tf-cpu:v1.12', 'username':'', 'password': ''}),
    #Name of the workspace in dkube. Update accordingly if different name is used while creating a workspace in dkube.
    training_program="mnist",
    #Script to run inside the training container    
    training_script="python model.py",
    tuning = '{ "studyName": "mnist-dkube-tuning", "owner": "crd", "optimizationtype": "maximize", "objectivevaluename": "train_accuracy_1", "optimizationgoal": 0.99, "requestcount": 1, "metricsnames": [ "train_accuracy_1" ], "parameterconfigs": [ { "name": "--learning_rate", "parametertype": "double", "feasible": { "min": "0.01", "max": "0.05" } }, { "name": "--batch_size", "parametertype": "int", "feasible": { "min": "100", "max": "200" } }, { "name": "--num_epochs", "parametertype": "int", "feasible": { "min": "1", "max": "10" } } ] }',
    #Input datasets for training. Update accordingly if different name is used while creating dataset in dkube.    
    training_datasets=json.dumps(["mnist"]),
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12    
    training_gpus=0,
    #Any envs to be passed to the training program    
    training_envs=json.dumps([{"steps": 100}]),
    #Device to be used for serving - dkube mnist example trained on gpu needs gpu for serving else set this param to 'cpu'
    serving_device='cpu'):

    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=training_datasets, ngpus=training_gpus,
                                    envs=training_envs, tuning=tuning)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact'], device=serving_device).after(train)
    inference   = dkube_viewer_op(auth_token, serving.outputs['servingurl'],
                                  'digits', viewtype='inference').after(serving)

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(d3pipeline, 'dkube_mnist_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

In [ ]:
run = client.run_pipeline(mnist_experiment.id, 'mnist_classifier_pipeline', 'dkube_mnist_pl.tar.gz', params={})